## Create a generative AI runbook to resolve security findings

## Module 2b

```
// Copyright Amazon.com, Inc. or its affiliates. All Rights Reserved.
// SPDX-License-Identifier: MIT-0
```

### Introduction

In this module, you will learn the art-of-the-possible and dive deep into some of the advanced features of Claude 3.
- Create a tool for Claude 3 to use
- Create a function to support when the tool is used
- Pass the results of the tool back into Claude 3
- Get the final result

**NOTE**: You can use SHIFT + ENTER to run each of the cells in this workbook.

With tool use, we substitute tool or function results into prompts. Claude 3 can't literally call or access tools and functions. Instead, we have Claude 3:

- Output the tool name and arguments it wants to call
- Halt any further response generation while the tool is called
- Then we reprompt with the appended tool results

Function calling is useful because it expands Claude 3's capabilities and enables Claude 3 to handle much more complex, multi-step tasks. 

We will use tool use / function calling to expand Claude 3's knowledge of Security Hub findings.

----

**NOTE**: This module is the same as module 2, but uses the Converse API. On May 30th, Anthropic released an update to improve the developer experience when using Claude 3 models tools.

#### Step 1: Create a tool for Claude 3 to use

In this step, we will setup our function to access Claude 3 and create a tool.

In [ ]:
%pip install -qU pip
%pip install -qUr requirements.txt

In [ ]:
import boto3
import os
import sys
import re
import json

modelId = "anthropic.claude-3-sonnet-20240229-v1:0"
aws_region = "us-west-2"

bedrock_client = boto3.client(service_name='bedrock-runtime', region_name=aws_region)
inference_config = {"temperature": 0}

def get_completion(messages, system_prompt=None, toolConfig=None):
    
    # Create the converse method parameters
    converse_api_params = {
        "modelId": modelId,
        "messages": messages,
        "inferenceConfig": inference_config,
    }
    
    # Check if system_text is provided
    if system_prompt:
        # If system_text is provided, add the system parameter to the converse_params dictionary
        converse_api_params["system"] = [{"text": system_prompt}]
        
    # Check if tool is provided
    if toolConfig:
        converse_api_params["toolConfig"] = toolConfig

    response = bedrock_client.converse(**converse_api_params)

    return response

Next, we define a tool that Claude 3 has access to.

The function is called `get_findings_by_severity` and takes two parameters:
- `severity` - the severity of the findings, which could be CRITICAL, HIGH or MEDIUM.
- `Limit` - the number of findings to return.

In [ ]:
toolConfig = {
    "tools": [
        {
          "toolSpec": {
            "name": "get_findings_by_severity",
            "description": "Lookup function for getting findings from Security Hub. Supports CRITICAL, HIGH, and MEDIUM severity.",
            "inputSchema": {
              "json": {
                "type": "object",
                "properties": {
                  "severity": {
                    "type": "string",
                    "description": "Severity level of the findings to return. This should be CRITICAL, HIGH, or MEDIUM. They must be uppercase"
                  },
                  "limit": {
                    "type": "string",
                    "description": "This parameter determine how many results to return"
                  }
                },
                "required": [
                  "severity", "limit"]
              }
            }
          }
        }
     ]
}

We then can create a prompt, and pass in the list of tools Claude 3 has access to. In our example, we want to find 3 medium findings in Security Hub. Claude 3 has access to one tool, get_findings_by_severity.

In [ ]:
user_prompt = {"role": "user", "content": [{"text": "What are 3 medium findings in Security Hub?"}]}

messages = [user_prompt]

print(f"User prompt: {user_prompt}")
response = get_completion(messages, toolConfig=toolConfig)
print(f"Stop reason: {response['stopReason']}")

assistant_response = response['output']['message']['content']

for item in assistant_response:
    print(item)
    if 'toolUse' in item:
        tool = item['toolUse']
        

#### Step 2: Create a function to support when the tool is used

Claude 3 outputed that it would like to use the new tool that we created `get_findings_by_severity`. In this step we will define it.

Just like in Module 1, we use boto3 to make an API call to Security Hub.

In [ ]:
import boto3
sh = boto3.client('securityhub')

def get_findings_by_severity(severity, limit):
    response = sh.get_findings(
        Filters={
                'SeverityLabel': [
                    {
                        'Value': severity,
                        'Comparison': 'EQUALS'
                    }
                ]
        },
        MaxResults=limit
    )
    return response['Findings']

In step 1, Claude 3 outputted that it would like to use a tool and provided the inputs.

```
{
    'toolUse': {
        'toolUseId': 'tooluse_J-3JCS6FSOmXk2OBepbiGg',
        'name': 'get_findings_by_severity',
        'input': {
            'severity': 'MEDIUM',
            'limit': '3'
        }
}
```

In this step, we will extract them, and call the get_findings_by_severity python function.

In [ ]:
# Extract the parameters from Claude 3 and call our new function
severity = tool['input']['severity']
limit = tool['input']['limit']

if severity and limit:
    sh_result = get_findings_by_severity(str(severity), int(limit))
    print("---------------- RESULT ----------------")
    print(f"{sh_result}")

#### Step 3: Pass the results of the tool back into Claude 3

The output of the function is a JSON representation of the findings, just like we got in module 1. Claude 3 expects the result of the function to be a string in a user message.

In [ ]:
# Create a message with the results from the Security Hub function into a user prompt.

tool_call = {"role": "assistant", "content": assistant_response}

tool_result = {
    "role": "user",
    "content": [
        {
            "toolResult": {
                "toolUseId": tool['toolUseId'],
                "content": [
                    {
                        "text": json.dumps(sh_result)
                        }
                ]
            }
        }
    ]
}

print(f"User prompt: {user_prompt}")
print(f"\nTool call: {tool_call}")
print(f"\nTool results: {tool_result}")


#### Step 4: Get the final result

In this step, we will combine the first user prompt, the tool call prompt, and the results of the tool (function) into a single message. This will return the final result.

In [ ]:
# Send back to Claude 3 by appending the result back to the message chain
messages = [user_prompt, tool_call, tool_result]

# Print Claude 3's response
final_response = get_completion(messages=messages, toolConfig=toolConfig)
print("------------- FINAL RESULT -------------")
print(final_response['output']['message']['content'][0]['text'])